In [16]:
%%capture
#!pip install intake xarray[complete] pystac

In [17]:
import pandas as pd
import xarray  as xr
import pystac
import json
from datetime import datetime,timezone

In [18]:
datetime(2020,1,1)

datetime.datetime(2020, 1, 1, 0, 0)

In [19]:
chirpsS3 = "s3://climate-action-datalake/zone=raw/source=chirps/variable=precipitation.zarr/"
datetime_utc = datetime.now(tz=timezone.utc)


In [20]:
catalog = pystac.Catalog(id='datacube-catalog', description='Datacube catalog stored in S3')

In [21]:
print(list(catalog.get_all_items()))
print(list(catalog.get_children()))

[]
[]


In [22]:
extent = pystac.Extent(spatial= pystac.SpatialExtent(bboxes=[-49.98,-180,49.98,180])
                       ,temporal=pystac.TemporalExtent(intervals=[datetime(1980,1,1),None]))

In [23]:
collection = pystac.Collection(id='chirps',
                               description='Data from chirps',
                               extent=extent)

In [24]:


dataset_item = pystac.Item(
    id="precipitation",
    geometry= None,
    bbox=None,
    datetime =datetime_utc,
    properties={
        "zarr_store": chirpsS3  # Reference the Zarr store location
    }
)

In [25]:
dataset_item.add_asset(
    key='chips-zarr',
    asset=pystac.Asset(
        title = "Data cube chirps",
        description = "Historical chirps global data transformed into cloud native format Zarr",
        href=chirpsS3,
        media_type=pystac.MediaType.ZARR ,
        extra_fields  = {
            'authentication_enabled': 'yes'
            ,'source': 'https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05'
            ,'update_frecuency': 'Monthy'
            ,'resolution': '0.05 x 0.05'
            ,'dimensions':
                [{'lat':'latitud'
                     ,'description': 'latitud of the data'},
                {'lon':'longitud'
                     ,'description': 'longitud of the data'},
                {'time':'date'
                     ,'description': 'The date of the registry at daily level'
                     ,'format':'yyyy-mm-dd'
                }]
            ,'variables':[{'precipitation':'precipitation'
                     ,'description': 'Precipitation of the geo point per date'
                     ,'units':'mm'}]
            ,'transformations': "From geotiff to zarr"
        }))


In [26]:
collection.add_items([dataset_item])

[<Link rel=item target=<Item id=precipitation>>]

In [27]:
catalog.add_child(collection)

<Link rel=child target=<Collection id=chirps>>

In [28]:
#catalog.add_item(dataset_item)

In [29]:
# dataset_item.get_parent()
#catalog.add_child(collection)

In [30]:
catalog.describe()

* <Catalog id=datacube-catalog>
    * <Collection id=chirps>
      * <Item id=precipitation>


In [31]:
help(dataset_item.add_asset)

Help on method add_asset in module pystac.asset:

add_asset(key: 'str', asset: 'Asset') -> 'None' method of pystac.item.Item instance
    Adds an Asset to this object.
    
    Args:
        key : The unique key of this asset.
        asset : The Asset to add.



In [32]:
catalog.normalize_and_save(root_href='../catalog', catalog_type = pystac.CatalogType.SELF_CONTAINED)

In [33]:
help(pystac.MediaType)

Help on class MediaType in module pystac.media_type:

class MediaType(pystac.utils.StringEnum)
 |  MediaType(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  A list of common media types that can be used in STAC Asset and Link metadata.
 |  
 |  Method resolution order:
 |      MediaType
 |      pystac.utils.StringEnum
 |      builtins.str
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  COG = image/tiff; application=geotiff; profile=cloud-optimized
 |  
 |  FLATGEOBUF = application/vnd.flatgeobuf
 |  
 |  GEOJSON = application/geo+json
 |  
 |  GEOPACKAGE = application/geopackage+sqlite3
 |  
 |  GEOTIFF = image/tiff; application=geotiff
 |  
 |  HDF = application/x-hdf
 |  
 |  HDF5 = application/x-hdf5
 |  
 |  HTML = text/html
 |  
 |  JPEG = image/jpeg
 |  
 |  JPEG2000 = image/jp2
 |  
 |  JSON = application/json
 |  
 |  KML = application/vnd.google-earth.kml+xml
 |  
 |  PDF = application/pdf
 |  


In [34]:
chirps = xr.open_zarr(chirpsS3)

c:\Users\cagonzalez\Anaconda3\envs\zarrKernel\lib\site-packages\fsspec\registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


NoCredentialsError: Unable to locate credentials

In [ ]:
chirps

<xarray.Dataset> Size: 906GB
Dimensions:        (band: 1, lat: 2000, lon: 7200, time: 15736)
Coordinates:
  * band           (band) int64 8B 0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time           (time) datetime64[ns] 126kB 1981-01-01 ... 2024-01-31
Data variables:
    precipitation  (time, band, lat, lon) float32 906GB dask.array<chunksize=(1, 1, 2000, 7200), meta=np.ndarray>